## 1. Controller.py
    - extractor, transformer 등 개별 모듈들에 대하여 순서대로 명령을 내려주는 Main 파일

In [ ]:
def main():
    "1. DB 커넥터 생성"
    
    "2. queries에서 테이블 이름 목록(table_list) 받아오기"
    
    for tbl in table_list:
        "3. extract >> DB 조회 후 dataframe 형태로 변환"
        
        "4. transform >> 저장 경로 생성 후 임시 저장 디렉토리 아래에 dataframe 저장"
        
        "5. load >> 저장소에 dataframe 파일 저장"
    
    "6. remove >> 저장이 끝난 후 임시 저장 디렉토리 삭제"

In [1]:
# 필요한 모듈들 import

from db.connector import DBconnector
from db.postgresql_query import queries
# from db.mysql_query import queries
from settings import DB_SETTINGS, TEMP_PATH
from pipeline.extract import extractor
from pipeline.transform import transformer
from pipeline.load import loader
from pipeline.remove import remover

In [2]:
# main controller 함수 작성

batch_date = "20230416"

def main(batch_date):
    
    db_connector = DBconnector(**DB_SETTINGS['POSTGRES'])
    
    for table_name in queries:
        print(table_name)
        pandas_df = extractor(db_connector, table_name, batch_date)
        res = transformer(batch_date, pandas_df, table_name)
        print(res)
        if res:
            
            db_connector = DBconnector(**DB_SETTINGS['POSTGRES'])
            
            loader(db_connector, pandas_df, table_name)
            
    # remover(TEMP_PATH)
            

main(batch_date)
    

fake_user
EXTRACTOR
DB에 접속합니다.
DB 연결을 끊습니다.


d:\KDT_Lecture\Lecture\day3\pipeline\extract.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(_query, connected.conn)


## 2. start.py

    - 모듈의 시작점이자, controller 함수에 배치 날짜를 전달해주는 파일

### Click 모듈
- CLI로 옵션이나 인자를 넘겨서 python 스크립트를 실행할 수 있도록 도와주는 라이브러리
- [CLICK DOCS](https://click.palletsprojects.com/en/8.1.x/options/#)

In [ ]:
import click

@click.command()
@click.option('-s', '--string-to-echo')
def click_test(string_to_echo):
    print(string_to_echo)

In [4]:
from datetime import datetime, timedelta, timezone
import click

@click.command
@click.option('-d', '--custom-batch-date', type=click.STRING, default='', help='배치작업연월일')
def start_batch(custom_batch_date):
    
    if custom_batch_date:
        batch_date = custom_batch_date
    else:
        batch_date = (datetime.now(timezone(timedelta(hours=9))) - timedelta(days = 1)).strftime('%Y%m%d')
        
    return batch_date

In [14]:
from datetime import datetime, timedelta, timezone
import click
from pipeline import controller

@click.command
@click.option('-d', '--custom-batch-date', type=click.STRING, default='', help='배치작업연월일')

def start_batch(custom_batch_date):
    
    print("START BATCH")
    
    if custom_batch_date:
        batch_date = custom_batch_date
    else:
        batch_date = (datetime.now(timezone(timedelta(hours=9))) - timedelta(days = 1)).strftime('%Y%m%d')
        
    controller.main(batch_date)

if __name__ == "__main__":
    start_batch()

first
EXTRACTOR
DB에 접속합니다.
DB 연결을 끊습니다.


d:\KDT_Lecture\Lecture\day3\pipeline\extract.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(_query, connected.conn)


## 3. 타기종 DB에 Connect (MYSQL)

##### 1)  MYSQL Server 다운로드
- 설치
> https://dev.mysql.com/downloads/file/?id=523568
<img src="https://velog.velcdn.com/images/newnew_daddy/post/9e356258-2a6b-4edf-aa5d-54ab83e25eef/image.png" width="80%">

- DBeaver Connect
> https://deeplify.dev/database/troubleshoot/public-key-retrieval-is-not-allowed
<img src="https://velog.velcdn.com/images/newnew_daddy/post/b8229e6b-f54b-4b2f-af9d-5cd03c703537/image.png" width="80%">

##### 1) 커넥션 테스트

In [1]:
# pymysql 라이브러리

import pymysql

host = 'localhost'
port = 3306
user = 'hyunsoo'
passwd = '910506'
db_name = 'faker'

db = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db_name, charset='utf8')

In [ ]:
# sqlalchemy 라이브러리

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine_name = 'mysql+pymysql'
user_id = 'hyunsoo'
user_pw = '910506'
host = 'localhost'
ip = '3306'
db_name = 'faker'   

db_orm = create_engine(f'{engine_name}://{user_id}:{user_pw}@{host}:{ip}/{db_name}')

##### 2) 데이터 넣기

In [ ]:
import os

for i in os.listdir('D:\KDT_Lecture\Lecture\mysql_data'):
    db_orm = create_engine(f'{engine_name}://{user_id}:{user_pw}@{host}:{ip}/{db_name}')
    
    root = 'D:\KDT_Lecture\Lecture\mysql_data'
    path = os.path.join(root, i)
    tbl_name = i.replace('.csv', '')
    print(tbl_name)
    df = pd.read_csv(path)
    df.to_sql(name=tbl_name, con=db_orm, if_exists='replace')

## 4. DBConnector.py에 MYSQL Connector 통합 실습

1. MYSQL Connection 함수 생성

2. DBConnector.py 파일 및 .env 파일에 MYSQL 관련 내용 반영

3. `DBconnector(**DB_SETTINGS['MYSQL'])` 코드를 통해 connection 생성